# Main analysis on api collected data to follow

In [20]:
# import libraries
"""
from eod import *
import os
import requests
import pandas as pd
import json
from pathlib import Path
import numpy as np
import datetime
import dateutil.parser
import urllib.request
from dotenv import load_dotenv
#import data viz libraries
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
#import altair as alt
%matplotlib inline
# import interactive panels libraries
import panel as pn
from panel.interact import interact
from panel import widgets
# pn.extension('plotly')
pn.extension()
"""
# import custom functions in same dir
from eod import *
# import main libraries
import os
import requests
import pandas as pd
import json
from pathlib import Path
import numpy as np
from datetime import datetime
import dateutil.parser
import urllib.request
from dotenv import load_dotenv
# import data viz libraries
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, CDSView, GroupFilter, CustomJS, MultiSelect, CheckboxGroup
from bokeh.models.widgets import Tabs, Panel
%matplotlib inline
# import interactive panels libraries
import panel as pn
from panel.interact import interact
from panel import widgets
# pn.extension('plotly')
pn.extension()

In [21]:
# Read the API keys
load_dotenv()

True

In [22]:
##########################################################
# Create data structures used throughout
##########################################################
# Define positions (indexes, algorithmitic portfolios, assets to evaluate
# Define currencies to review performance against
# Define evaluation calculations to perform
# All are manually coded in for first MVP

asset1 = 'SP500'
asset2 = 'SP500BDT'
asset3 = 'FDN'
asset4 = 'VNQ'
asset5 = 'GlobalArtIndex'
asset6 = 'XAUUSD'
asset7 = 'BTCUSD'

currency1 = 'USD'
currency2 = 'XAU'
currency3 = 'BTC'

In [23]:
####################################################################
# Main Market Dataset Collection
# Equities
# Precious Metals
# Crpyto currencies
####################################################################
to_eod_symbol_dict = {
                    'SP500BDT' : 'SP500BDT.INDX',
                    'SP500' : 'GSPC.INDX',
                    'BTCUSD' : 'BTC-USD.CC',
                    'XAUUSD' : 'GC.COMM',
                    'VNQ' : 'VNQ.US',
                    'FDN' : 'FDN.US'}
#from_eod_symbol_dict = {}
#for key, value in to_eod_symbol_dict.items():
#    from_eod_symbol_dict[value] = key
#print(to_eod_symbol_dict[asset1])

start_date = '2010-01-01'
asset1_data_USD_df = get_data(to_eod_symbol_dict[asset1],start_date)
print(asset1)
asset1_data_USD_df['Asset'] = asset1
#asset1_data_USD_df['Asset'] = asset1_data_USD_df['Asset'].replace([to_eod_symbol_dict[asset1],'SP500'])
#asset1_data_USD_df['Asset'].replace([to_eod_symbol_dict[asset1],asset1],inplace=True)
asset1_data_USD_df.head(2)

SP500


,Asset,AdjClosePrice
Date,,
2010-01-04,SP500,1132.99
2010-01-05,SP500,1136.52


In [24]:
asset2_data_USD_df = get_data(to_eod_symbol_dict[asset2],start_date)
print(asset2)
asset2_data_USD_df['Asset'] = asset2
asset2_data_USD_df.head(2)

SP500BDT


,Asset,AdjClosePrice
Date,,
2010-01-01,SP500BDT,288.90
2010-01-04,SP500BDT,289.37


In [25]:
asset3_data_USD_df = get_data(to_eod_symbol_dict[asset3],start_date)
print(asset3)
asset3_data_USD_df['Asset'] = asset3
asset3_data_USD_df.head(2)

FDN


,Asset,AdjClosePrice
Date,,
2010-01-04,FDN,25.5341
2010-01-05,FDN,25.5141


In [26]:
asset4_data_USD_df = get_data(to_eod_symbol_dict[asset4],start_date)
print(asset4)
asset4_data_USD_df['Asset'] = asset4
asset4_data_USD_df.head(2)

VNQ


,Asset,AdjClosePrice
Date,,
2010-01-04,VNQ,29.1221
2010-01-05,VNQ,29.0892


In [27]:
asset6_data_USD_df = get_data(to_eod_symbol_dict[asset6],start_date)
print(asset6)
asset6_data_USD_df['Asset'] = asset6
currency2_data_df = asset6_data_USD_df.copy()
asset6_data_USD_df.head(2)

XAUUSD


,Asset,AdjClosePrice
Date,,
2010-01-03,XAUUSD,1095.2
2010-01-04,XAUUSD,1118.3


In [28]:
asset7_data_USD_df = get_data(to_eod_symbol_dict[asset7],start_date)
print(asset7)
asset7_data_USD_df['Asset'] = asset7
currency3_data_df = asset7_data_USD_df.copy()
asset7_data_USD_df.head(2)


BTCUSD


,Asset,AdjClosePrice
Date,,
2010-07-17,BTCUSD,0.0495
2010-07-18,BTCUSD,0.1000


In [29]:
## Fetch ART Data ##
art_file_path = Path("../data_imports/Artprice_Indexes_Quarterly_data_Base100_January1998.csv")
art_indexes_df = pd.read_csv(art_file_path, encoding='utf-8', parse_dates=True, infer_datetime_format=True)
art_indexes_df.rename(columns=({"Unnamed: 0" : "Date"}), inplace=True)
art_indexes_df['Date'] =  pd.to_datetime(art_indexes_df['Date'], format='%Y-%m-%d')
# start massaging master art dataset for global index before separating global art index out
art_indexes_df['AdjClosePrice'] = ((art_indexes_df.loc[:,'Global Index (USD)'])*100).astype(float)
art_indexes_df['Asset'] = 'GlobalArtIndex'
# now true copy just whats needed and finish massage
gblart_index_df = art_indexes_df[["Date", "Asset", "AdjClosePrice"]].copy()
gblart_index_df.set_index("Date", drop=True, inplace=True)
gblart_index_df.dropna()

asset5_data_USD_df = gblart_index_df
print(asset5_data_USD_df.head())

                     Asset  AdjClosePrice
Date                                     
1998-01-01  GlobalArtIndex        10000.0
1998-04-01  GlobalArtIndex        10700.0
1998-07-01  GlobalArtIndex        10600.0
1998-10-01  GlobalArtIndex         9600.0
1999-01-01  GlobalArtIndex        10400.0


In [30]:
####################################################################
# BELOW STARTS THE FINAL VERSION OF ANALYTICS AND VISUALIZATIONS
####################################################################

# Define calculation, currency and asset structure for 
# currency conversion and main analysis

# hard code for now to test

calc_list = ['STD_200D','MA_200D','DailyReturns','CumulReturns']


In [31]:
####################################################################
# CURRENCY CONVERSION FUNCTION 
####################################################################
def convert_currency(currency,wip_convert_df):
    """
    Convert market price data priced in USD to currency indicated. 
    Initially support price conversion USD->XAU, USD->BTC
    """
    if currency == "XAU":
        print(f"\tConverting price USD -> {currency}.")
        wip_convert_combo_df = wip_convert_df.merge(currency2_data_df,how='left',left_index=True,right_index=True).dropna()
        wip_convert_combo_df['AdjClosePrice'] = (wip_convert_combo_df['AdjClosePrice_x']) / (wip_convert_combo_df['AdjClosePrice_y'])
        wip_convert_combo_df.rename(columns=({"Asset_x" : "Asset"}), inplace=True)
        wip_convert_combo_df.drop(["AdjClosePrice_x", "Asset_y", "AdjClosePrice_y"], inplace=True, axis = 1)
    elif currency == "BTC":
        print(f"\tConverting price USD -> {currency}..")
        wip_convert_combo_df = wip_convert_df.merge(currency3_data_df,how='left',left_index=True,right_index=True).dropna()
        wip_convert_combo_df['AdjClosePrice'] = (wip_convert_combo_df['AdjClosePrice_x']) / (wip_convert_combo_df['AdjClosePrice_y'])
        wip_convert_combo_df.rename(columns=({"Asset_x" : "Asset"}), inplace=True)
        wip_convert_combo_df.drop(["AdjClosePrice_x", "Asset_y", "AdjClosePrice_y"], inplace=True, axis = 1)
    else:
        print(f"What is {currency} doing here?!")
    return(wip_convert_df)


In [32]:
####################################################################
# ASSET ANALYSIS FUNCTION 
####################################################################
def analyze_asset(calc_list,wip_calc_df):
    """
    Market price data analysis and asset return calcs
    as defined in calc_list input on provided wip_calc_df
    of format: date, asset, adjclose
    """
    print(f"\t\tAnalyzing price and returns...")
    for calc_type in calc_list:
        # Do analysis on price trends before return calcs 
        # to align return calcs with same window
        if calc_type == 'STD_200D':
            wip_calc_df[calc_type] = wip_calc_df['AdjClosePrice'].rolling(window=200).std()
        elif calc_type == 'MA_200D':
            wip_calc_df[calc_type] = wip_calc_df['AdjClosePrice'].rolling(window=200).mean()
            wip_calc_df.dropna(inplace=True)
        elif calc_type == 'DailyReturns':
            wip_calc_df[calc_type] = wip_calc_df['AdjClosePrice'].pct_change()
            wip_calc_df.dropna(inplace=True)
        elif calc_type == 'CumulReturns':
            wip_calc_df[calc_type] = (1+wip_calc_df['DailyReturns']).cumprod() - 1
        else:
            print(f"What is {calc_type} doing here?!")        
    return(wip_calc_df)


In [33]:
####################################################################
# MAIN ANALYSIS LOOP 
####################################################################
# hard code for now
# Asset1
print("\nProcessing Asset1....")
#for USD
wip_calc_df = asset1_data_USD_df.copy()
asset1_calc_USD_df = analyze_asset(calc_list,wip_calc_df)
# for BTC
wip_convert_df = asset1_data_USD_df.copy()
wip_calc_df = convert_currency('BTC',wip_convert_df)
asset1_calc_BTC_df = analyze_asset(calc_list,wip_calc_df)
# for XAU
wip_convert_df = asset1_data_USD_df.copy()
wip_calc_df = convert_currency('XAU',wip_convert_df)
asset1_calc_XAU_df = analyze_asset(calc_list,wip_calc_df)

# Asset2
print("\nProcessin Asset2....")
#for USD
wip_calc_df = asset2_data_USD_df.copy()
asset2_calc_USD_df = analyze_asset(calc_list,wip_calc_df)
# for BTC
wip_convert_df = asset2_data_USD_df.copy()
wip_calc_df = convert_currency('BTC',wip_convert_df)
asset2_calc_BTC_df = analyze_asset(calc_list,wip_calc_df)
# for XAU
wip_convert_df = asset2_data_USD_df.copy()
wip_calc_df = convert_currency('XAU',wip_convert_df)
asset2_calc_XAU_df = analyze_asset(calc_list,wip_calc_df)

# Asset3
print("\nProcessin Asset3....")
#for USD
wip_calc_df = asset3_data_USD_df.copy()
asset3_calc_USD_df = analyze_asset(calc_list,wip_calc_df)
# for BTC
wip_convert_df = asset3_data_USD_df.copy()
wip_calc_df = convert_currency('BTC',wip_convert_df)
asset3_calc_BTC_df = analyze_asset(calc_list,wip_calc_df)
# for XAU
wip_convert_df = asset3_data_USD_df.copy()
wip_calc_df = convert_currency('XAU',wip_convert_df)
asset3_calc_XAU_df = analyze_asset(calc_list,wip_calc_df)

# Asset4
print("\nProcessin Asset4....")
#for USD
wip_calc_df = asset4_data_USD_df.copy()
asset4_calc_USD_df = analyze_asset(calc_list,wip_calc_df)
# for BTC
wip_convert_df = asset4_data_USD_df.copy()
wip_calc_df = convert_currency('BTC',wip_convert_df)
asset4_calc_BTC_df = analyze_asset(calc_list,wip_calc_df)
# for XAU
wip_convert_df = asset4_data_USD_df.copy()
wip_calc_df = convert_currency('XAU',wip_convert_df)
asset4_calc_XAU_df = analyze_asset(calc_list,wip_calc_df)

# Asset5
print("\nProcessin Asset5....")
#for USD
wip_calc_df = asset5_data_USD_df.copy()
asset5_calc_USD_df = analyze_asset(calc_list,wip_calc_df)
# for BTC
wip_convert_df = asset5_data_USD_df.copy()
wip_calc_df = convert_currency('BTC',wip_convert_df)
asset5_calc_BTC_df = analyze_asset(calc_list,wip_calc_df)
# for XAU
wip_convert_df = asset5_data_USD_df.copy()
wip_calc_df = convert_currency('XAU',wip_convert_df)
asset5_calc_XAU_df = analyze_asset(calc_list,wip_calc_df)

# Asset6
print("\nProcessin Asset6....")
#for USD
wip_calc_df = asset6_data_USD_df.copy()
asset6_calc_USD_df = analyze_asset(calc_list,wip_calc_df)
# for BTC
wip_convert_df = asset6_data_USD_df.copy()
wip_calc_df = convert_currency('BTC',wip_convert_df)
asset6_calc_BTC_df = analyze_asset(calc_list,wip_calc_df)

# Asset7
print("\nProcessin Asset7....")
#for USD
wip_calc_df = asset7_data_USD_df.copy()
asset6_calc_USD_df = analyze_asset(calc_list,wip_calc_df)
# for XAU
wip_convert_df = asset7_data_USD_df.copy()
wip_calc_df = convert_currency('XAU',wip_convert_df)
asset7_calc_XAU_df = analyze_asset(calc_list,wip_calc_df)



Processing Asset1....
		Analyzing price and returns...
	Converting price USD -> BTC..
		Analyzing price and returns...
	Converting price USD -> XAU.
		Analyzing price and returns...

Processin Asset2....
		Analyzing price and returns...
	Converting price USD -> BTC..
		Analyzing price and returns...
	Converting price USD -> XAU.
		Analyzing price and returns...

Processin Asset3....
		Analyzing price and returns...
	Converting price USD -> BTC..
		Analyzing price and returns...
	Converting price USD -> XAU.
		Analyzing price and returns...

Processin Asset4....
		Analyzing price and returns...
	Converting price USD -> BTC..
		Analyzing price and returns...
	Converting price USD -> XAU.
		Analyzing price and returns...

Processin Asset5....
		Analyzing price and returns...
	Converting price USD -> BTC..
		Analyzing price and returns...
	Converting price USD -> XAU.
		Analyzing price and returns...

Processin Asset6....
		Analyzing price and returns...
	Converting price USD -> BTC..
		A

In [34]:
# Verify no original dataset corruption
print(asset1_data_USD_df.head(2))

            Asset  AdjClosePrice
Date                            
2010-01-04  SP500        1132.99
2010-01-05  SP500        1136.52


In [35]:
####################################################################
# THIS CELL IS START OF VISUALIZATION FUNCTIONS 
#
####################################################################
# Define Panel Visualization Functions
# Number of plot functions right now = 5 calcs/position * 2timeframes * 6 positions = 60!!!!

#def 
#art_index_std_bar = px.bar(art_index_std, title="Art Price STD 1998-2020")
#art_index_std_bar

In [36]:
# Define Multi Select Interaction

def multi_select_assets():
    options = [("1", "S&P 500"), ("2", "S&P500 Bond Index"), ("3", "Coming Soon"), ("4", "Coming Soon"), ("5", "Global Art Index"), ("6", "Gold"), ("7", "BTC USD")]
    
    multi_select = MultiSelect(value=["1", "2"], options=options)
    multi_select.js_on_change("value", 
                              CustomJS(
        code="""
        console.log('multi_select: value=' + this.value, this.toString())
        """)
                             )
    return multi_select
   
def multi_select_currency():
    labels = ["USD", "Gold (XAU)", "BTC"]
    
    checkbox_group = CheckboxGroup(labels=labels, active=[0])
    checkbox_group.js_on_click(CustomJS(code="""
    console.log('checkbox_group: active=' + this.active, this.toString())
    """))

    return checkbox_group


# Define Plot Functions 

def price_performance(df):
    
    # Output the visualization directly in the notebook
    output_notebook()
    
        
    # Parse Column data
    source = ColumnDataSource(data=df)
    
    # Create the figure object
    pp_fig = figure(
        x_axis_type='datetime',
        title=("Price Performance"),
        plot_height=400, plot_width=900,
        x_axis_label='Date', y_axis_label='Returns',
    )
 
    # Draw the figure
  
    pp_fig.line(
        x='Date',
        y="AdjClosePrice",
        source=source,
        color='green', 
        width=0.75, 
        legend='Price'
    )
    
    pp_fig.line(
        x='Date',
        y="CumulReturns",
        source=source,
        color='purple', 
        width=0.75, 
        legend='Cumulative Returns'
    )
    
    pp_fig.line(
        x='Date',
        y="STD_200D",
        source=source,
        color='red', 
        width=0.75, 
        legend='200 Day STD'
    )
        
    pp_fig.line(
        x='Date',
        y="MA_200D",
        source=source,
        color='blue', 
        width=0.75, 
        legend='200 Day Moving Average'
    )
       
    pp_fig.legend.location = 'top_left'
    
    pp_pane = pn.pane.Bokeh(pp_fig)
    
    return pp_pane

def cum_returns(df):
    
    # Output the visualization directly in the notebook
    output_notebook()
    
        
    # Parse Column data
    source = ColumnDataSource(data=df)
    
    # Create the figure object
    cr_fig = figure(
        x_axis_type='datetime',
        title=("Cumulative Returns"),
        plot_height=400, plot_width=800,
        x_axis_label='Date', y_axis_label='Returns',
    )
 
    # Draw the figure
  
    cr_fig.line(
        x='Date',
        y="CumulReturns",
        source=source,
        color='red', 
        width=0.75, 
        legend='Price'
    )
    
    cr_fig.legend.location = 'top_left'
    
    cr_pane = pn.pane.Bokeh(cr_fig)
    
    return cr_pane
    

# Creating the Price Performance Panes

sp500_pp_pane = price_performance(asset1_calc_USD_df)
sp500_bond_pp_pane = price_performance(asset2_calc_USD_df)
sp500_pane = price_performance(asset3_calc_USD_df)
sp500_pane = price_performance(asset4_calc_USD_df)
art_pp_pane = price_performance(asset5_calc_USD_df)
gold_pp_pane = price_performance(asset6_calc_USD_df)
btc_pp_pane = price_performance(asset7_calc_USD_df)

# sp_500_pp_pane.title.text = 'Eastern Conference'

# Creating the Cumulative Returns Panes

sp500_cr_pane = cum_returns(asset1_calc_USD_df)
sp500_bond_cr_pane = cum_returns(asset2_calc_USD_df)
sp500_pane = cum_returns(asset3_calc_USD_df)
sp500_pane = cum_returns(asset4_calc_USD_df)
art_cr_pane = cum_returns(asset5_calc_USD_df)
gold_cr_pane = cum_returns(asset6_calc_USD_df)
btc_cr_pane = cum_returns(asset7_calc_USD_df)

In [36]:
# Create Panel Dashboard

welcome_column = pn.Column(
    "# On The Money Portfolio Management Tool",
    "### We aim to maximize retail investors’ alpha over the long term by helping them make more informed decisions.",
    
    
    "#### Welcome to the asset analysis dashboard. Use the tabs above to navigate through the different sections."
    "Use the input below to see how your ideas and assets compare with the benchmarks.",
    
    "### Select the Asset Classes you wish to view:",
    "##### *(Press and Hold Command to select multiple)*",
    multi_select_assets(),
    multi_select_currency()
)

price_performance = pn.Column(
    "### S&P 500",
    sp500_pp_pane,
    "### S&P 500 Bond Index",
    sp500_bond_pp_pane,
    "### Global Art Index",
    art_pp_pane,
    "### Gold",
    gold_pp_pane,
    "### Bitcoin",
    btc_pp_pane
)

cum_return = pn.Column(
    "### S&P 500",
    sp500_rc_pane,
    "### S&P 500 Bond Index",
    sp500_bond_cr_pane,
    "### Global Art Index",
    art_cr_pane,
    "### Gold",
    gold_cr_pane,
    "### Bitcoin",
    btc_cr_pane
)

portfolio_dashboard = pn.Tabs(
    (
        "Welcome",
        welcome_column
    ),
    (
        "Price Performance Of Asset Classes",
        price_performance
    ),
    (
        "Cumulative Return Of Asset Classes",
        cum_return
    )
)

portfolio_dashboard

In [36]:
portfolio_dashboard.servable()

In [ ]:
portfolio_dashboard.app('localhost:8888')